<a href="https://colab.research.google.com/github/mostafa-ja/graph-tutorial/blob/master/Illegal_Bitcoin_Transactions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install torch-geometric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.10.0+cu113.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.6/107.6 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 417, in run
    _, build_failures = build(
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/wheel_builder.py", line 320, in build
    wheel_file = _build_one(
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/wheel_builder.py", line 194, in 

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(rc={'axes.facecolor':'dimgrey', 'grid.color':'lightgrey'})

import numpy as np
import pandas as pd
import networkx as nx
import torch
import torch.nn.functional as F
import torch.nn as nn
#import torch_scatter
#from torch_geometric.data import Data
print(torch.__version__)

# # The PyG built-in GCNConv
# from torch_geometric.nn import GCNConv
#from torch_geometric.nn.conv import MessagePassing
#import torch_geometric.transforms as T
#from torch_geometric.utils import remove_self_loops, add_self_loops, softmax, degree

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,roc_auc_score
import scipy.sparse as scsp
from sklearn.cluster import KMeans
import copy

2.0.1+cu118


In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
url_data_e = (r'https://raw.githubusercontent.com/yuchenWYC/'
              r'Elliptic_dataset/master/edges.csv')
edges = pd.read_csv(url_data_e)

url_data_n = (r'https://media.githubusercontent.com/media/yuchenWYC/'
              r'Elliptic_dataset/master/nodes.csv')
nodes = pd.read_csv(url_data_n)

In [3]:
nodes.info

<bound method DataFrame.info of             txId  class  timestamp         2         3         4          5  \
0      232438397      0          1  0.163054  1.963790 -0.646376  12.409294   
1      232029206      0          1 -0.005027  0.578941 -0.091383   4.380281   
2      232344069      0          1 -0.147852 -0.184668 -1.201369  -0.121970   
3       27553029      0          1 -0.151357 -0.184668 -1.201369  -0.121970   
4        3881097      0          1 -0.172306 -0.184668 -1.201369   0.028105   
...          ...    ...        ...       ...       ...       ...        ...   
46559   80329479      0         49 -0.159293 -0.037276  1.018602  -0.121970   
46560  158406298      0         49 -0.172962 -0.126566  1.018602  -0.121970   
46561  158375075      1         49 -0.170412 -0.078164  1.018602   0.028105   
46562  147478192      0         49 -0.093732 -0.116160  1.018602  -0.121970   
46563  158375402      1         49 -0.172014 -0.078182  1.018602   0.028105   

              6    

In [4]:
nodes.iloc[0:2, 0:10]

,txId,class,timestamp,2,3,4,5,6,7,8
0,232438397,0,1,0.163054,1.963790,-0.646376,12.409294,-0.063725,9.782742,12.414558
1,232029206,0,1,-0.005027,0.578941,-0.091383,4.380281,-0.063725,4.667146,0.851305


In [5]:
edges.head()

,txId1,txId2
0,232344069,27553029
1,3881097,232457116
2,232051089,232470704
3,230473487,7089694
4,231182296,14660781


In [7]:
nodes.iloc[:, 2:]

,timestamp,2,3,4,5,6,7,8,9,10,...,157,158,159,160,161,162,163,164,165,166
0,1,0.163054,1.963790,-0.646376,12.409294,-0.063725,9.782742,12.414558,-0.163645,-0.115831,...,-0.577099,-0.613614,0.241128,0.241406,1.072793,0.085530,-0.131155,0.677799,-0.120613,-0.119792
1,1,-0.005027,0.578941,-0.091383,4.380281,-0.063725,4.667146,0.851305,-0.163645,-0.144554,...,-0.577099,-0.613614,0.241128,0.241406,0.604120,0.008632,-0.131155,0.333211,-0.120613,-0.119792
2,1,-0.147852,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.137933,-0.144108,...,-0.577099,-0.613614,0.241128,0.241406,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792
3,1,-0.151357,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.141519,-0.147643,...,-0.539735,-0.582077,-0.979074,-0.978556,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792
4,1,-0.172306,-0.184668,-1.201369,0.028105,-0.043875,-0.029140,0.242712,-0.163640,-0.169115,...,-0.577099,-0.600999,0.241128,0.241406,0.018279,-0.068266,-0.084674,-0.054450,-1.760926,-1.760984
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46559,49,-0.159293,-0.037276,1.018602,-0.121970,0.035526,-0.113002,-0.061584,-0.149635,-0.155646,...,1.793987,1.408971,0.231244,-0.388216,-0.098889,1.931078,3.168259,3.707301,-1.390548,-1.214035
46560,49,-0.172962,-0.126566,1.018602,-0.121970,-0.063725,-0.113002,-0.061584,-0.163622,-0.169437,...,-0.577099,0.647874,0.241128,0.241406,10.914916,1.700384,-0.131155,7.914145,-0.120613,-0.119792
46561,49,-0.170412,-0.078164,1.018602,0.028105,-0.043875,0.054722,-0.061584,-0.163631,-0.167106,...,1.709623,1.606604,1.461330,1.461369,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792
46562,49,-0.093732,-0.116160,1.018602,-0.121970,-0.043875,-0.113002,-0.061584,-0.082559,-0.089510,...,-0.577099,-0.613614,0.241128,0.241406,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792


In [8]:
nodes.iloc[:, 2]

0         1
1         1
2         1
3         1
4         1
         ..
46559    49
46560    49
46561    49
46562    49
46563    49
Name: timestamp, Length: 46564, dtype: int64

In [9]:

# Split the graph and store node features, edges (represented by adjacency list),
# and labels separately by timestamp t (from 1 to 49).


features =  torch.FloatTensor(nodes.iloc[:, 2:].to_numpy())
times = nodes.iloc[:, 2].to_numpy()
times = torch.LongTensor(times.reshape(len(times),))
labels = nodes.iloc[:, 1].to_numpy()
labels = torch.LongTensor(labels.reshape(len(labels),))

nodes_id = nodes.iloc[:, 0].to_numpy()
nodes_id = torch.LongTensor(nodes_id.reshape(len(nodes_id),))

min_t = torch.min(times) # 1
max_t = torch.max(times) # 49

# Construct nodes of the directed graph for each time step;
# features by time step are stored in "features_t"; labels by
# time step are stored in "labels_t"
features_t = [] #(49,(2147, 166))
labels_t = []  #(49,(2147))

# Create a dictionary where
# <key, value> = <node_id, <<idx, node_index_in_time_t_subgraph>, <t, time_t>>>.
id2idx = {}
for t in range(min_t, max_t + 1):
    features_t.append(features[times == t, :])
    labels_t.append(labels[times == t])
    nodes_t = nodes_id[times == t]
    for i in range(nodes_t.shape[0]):
        id2idx[nodes_t[i].item()] = {}
        id2idx[nodes_t[i].item()]['idx'] = i
        id2idx[nodes_t[i].item()]['t'] = t



In [16]:
len(labels_t)

49

In [17]:
labels_t[0].shape

torch.Size([2147])

In [10]:
id2idx 


{232438397: {'idx': 0, 't': 1},
 232029206: {'idx': 1, 't': 1},
 232344069: {'idx': 2, 't': 1},
 27553029: {'idx': 3, 't': 1},
 3881097: {'idx': 4, 't': 1},
 232457116: {'idx': 5, 't': 1},
 232412405: {'idx': 6, 't': 1},
 232470342: {'idx': 7, 't': 1},
 230550393: {'idx': 8, 't': 1},
 232051089: {'idx': 9, 't': 1},
 232470704: {'idx': 10, 't': 1},
 232033533: {'idx': 11, 't': 1},
 230473487: {'idx': 12, 't': 1},
 7089694: {'idx': 13, 't': 1},
 231179595: {'idx': 14, 't': 1},
 231177927: {'idx': 15, 't': 1},
 2758467: {'idx': 16, 't': 1},
 232437171: {'idx': 17, 't': 1},
 3878886: {'idx': 18, 't': 1},
 231182296: {'idx': 19, 't': 1},
 14660781: {'idx': 20, 't': 1},
 43358239: {'idx': 21, 't': 1},
 230528714: {'idx': 22, 't': 1},
 230585122: {'idx': 23, 't': 1},
 2881274: {'idx': 24, 't': 1},
 2881273: {'idx': 25, 't': 1},
 230516478: {'idx': 26, 't': 1},
 5488136: {'idx': 27, 't': 1},
 27553759: {'idx': 28, 't': 1},
 232467632: {'idx': 29, 't': 1},
 231181128: {'idx': 30, 't': 1},
 2324

In [ ]:
# Construct adjacency lists of the directed graph (non-symmetric) for each time step;
# adjacency lists for each time step are stored in "edge_indices".
edge_idx_t = [[] for _ in range(min_t, max_t + 1)]
for index in range(edges.shape[0]):   
    node1_t = id2idx[edges.iloc[index, 0]]['t']
    node1_idx = id2idx[edges.iloc[index, 0]]['idx']
    node2_t = id2idx[edges.iloc[index, 1]]['t']
    node2_idx = id2idx[edges.iloc[index, 1]]['idx']
    edge_idx_t[node1_t - 1].append([node1_idx, node2_idx]) # time_step starts from 1

edge_indices = [torch.LongTensor(edge_idx_t[i]).t() for i in range(len(edge_idx_t))]